In [65]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [66]:
# Load the dataset
data = pd.read_csv("cleaned_data/cleaned_asylum_seekers_added.csv")
data = data[data['HDI'] != '..']
data = data.dropna()
decimals = 2    
data['Unemployment rate'] = data['Unemployment rate'].apply(lambda x: round(x, decimals))
data['origin_to_target_dist'] = data['origin_to_target_dist'].apply(lambda x: round(x, decimals))


C:\Users\LM\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [67]:
col_names = [   'HDI','origin_to_target_dist','Encoded Target Country', 'Encoded procedure type'] # Declare the columns names

x = data[col_names] # 
y = data['accepted/rejected'] # Target Variable


In [68]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [69]:
dt = DecisionTreeRegressor(max_depth=5,
                           min_samples_leaf=0.13,
                           random_state=1)

dt.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.13,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [70]:
# Import mean_squared_error from sklearn.metrics as MSE
from sklearn.metrics import mean_squared_error as MSE

# Compute y_pred
y_pred = dt.predict(x_test)

# Compute mse_dt
mse_dt = MSE(y_test, y_pred)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {:.2f}".format(rmse_dt))

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))


df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

Test set RMSE of dt: 0.69
Mean Absolute Error: 0.46478964411017704
Mean Squared Error: 0.4724709906250392


,Actual,Predicted
54282,0,0.365555
74594,0,0.170165
96300,0,0.365555
44548,0,0.170165
80242,0,0.170165
62296,0,0.365555
42370,0,0.334707
9014,0,0.170165
85790,0,0.334707
63403,2,0.365555


In [21]:
params_dt = {
             'max_depth': [2, 3, 4],
             'min_samples_leaf': [0.12, 0.14, 0.16, 0.18]
            }

In [25]:
from sklearn.model_selection import GridSearchCV

# Instantiate grid_dt
grid_dt = GridSearchCV(estimator=dt,
                       param_grid=params_dt,
                       scoring='roc_auc',
                       cv=5,
                       n_jobs=-1)
grid_dt.fit(x_train, y_train )

ValueError: multiclass format is not supported

In [ ]:
# Import roc_auc_score from sklearn.metrics 
from sklearn.metrics import roc_auc_score

# Extract the best estimator
best_model = grid_dt.best_estimator_

# Predict the test set probabilities of the positive class
y_pred_proba = best_model.predict_proba(X_test)[:,1]

# Compute test_roc_auc
test_roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print test_roc_auc
print('Test set ROC AUC score: {:.3f}'.format(test_roc_auc))